In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv('../input/ipl-2020-player-performance/Training.csv')

In [3]:
train.head()

In [4]:
train['player_name'] = train['Id'].apply(lambda x: x.split("_")[-1])
train['player_id'] = train['Id'].apply(lambda x: x.split("_")[0])
train['player_id'] = train['player_id'].astype(int)

In [5]:
train.head()

In [6]:
train.info()

In [7]:
submission=pd.read_csv('../input/ipl-2020-player-performance/sample_submission.csv')

In [8]:
submission.head()

In [9]:
len(submission['Id'].unique())

In [10]:
matches_2020 = pd.read_csv('../input/ipl-2020-player-performance/Matches IPL 2020.csv')
matches_2020

In [11]:
matches_2020.info()

In [12]:
prev_matches=pd.read_csv('../input/ipl-2020-player-performance/Matches IPL 2008-2019.csv')

In [13]:
prev_matches.info()

In [14]:
team_players=pd.read_csv('../input/ipl-2020-player-performance/IPL 2020 Squads.csv',encoding='unicode_escape')

In [15]:
team_players

In [16]:
team_players.info()

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
plt.figure(figsize=(12,12),dpi=200)
sns.heatmap(train.corr(),annot =True)

In [19]:
df=pd.DataFrame()
df['player_Id']=train['player_id']
df['player_name'] = train['player_name']
df['total_score']=train['Total Points']
df['Id']=train['Id']

In [20]:
df.info()

In [21]:
df.head()

In [22]:
df = df.drop(['player_name'],axis=1)

In [23]:
df.head()

In [24]:
df=df.iloc[:1283]

In [25]:
df.shape

In [26]:
d =df.select_dtypes(include='object')
n=df.select_dtypes(exclude='object')

In [27]:
n

In [28]:
d=pd.get_dummies(d,drop_first=True)

In [29]:
df=pd.concat([d,n],axis=1)
df

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [31]:
X= df.drop(['total_score'],axis=1)
y= df['total_score']

In [32]:
X.shape

In [33]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [34]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_train.shape

In [35]:
X_test = scaler.transform(X_test)

In [36]:
import xgboost
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [37]:
clfs = {
    'Random Forest': RandomForestRegressor(n_estimators=500),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'Lasso': Lasso(alpha=100),
    'xgboost': xgboost.XGBRegressor(objective ='reg:linear', 
                  n_estimators = 10, seed = 123),
    'LinearRegression':LinearRegression(),
}

In [38]:
for i, (name, clf) in enumerate(clfs.items()):
    if name == 'LinearRegression':
        lr = pd.Series(np.round(clf.fit(X_train, y_train).predict(X_test)))
    elif name == 'DecisionTreeRegressor':
        dtr = pd.Series(np.round(clf.fit(X_train, y_train).predict(X_test)))
    elif name == "Random Forest":
        randomforest = pd.Series(np.round(clf.fit(X_train, y_train).predict(X_test)))
    elif name == "Lasso":
        lasso = pd.Series(np.round(clf.fit(X_train, y_train).predict(X_test)))
    elif name == "xgboost":
        xgbr = pd.Series(np.round(clf.fit(X_train, y_train).predict(X_test)))

In [39]:
preds = pd.concat([lr, dtr, randomforest, lasso, xgbr], axis=1, keys=['lr','dtr', 'randomforest', 'lasso', 'xgbr'])

In [40]:
preds

In [41]:
for i in preds.columns:
    print('RMSE of %s model is %f' % (i,np.sqrt(mean_squared_error(y_test,preds[i]))))

In [42]:
submission.head()

In [43]:
submission['player_name'] = submission['Id'].apply(lambda x: x.split("_")[-1])
submission['player_id'] = submission['Id'].apply(lambda x: x.split("_")[0])
submission['player_id'] = submission['player_id'].astype(int)

In [44]:
X_submission = submission.drop(['Total Points','player_name'],axis=1)
X_submission.player_id = X_submission.player_id.astype(int)
X_submission.player_id.dtypes

In [45]:
X_submission.info()

In [46]:
x = X_submission.select_dtypes(include='object') 
s = X_submission.select_dtypes(exclude='object')

x.columns
x= pd.get_dummies(x,drop_first=True)
x.columns

In [47]:
X_submission = pd.concat([x,s],axis=1)

X_submission.head()

In [48]:
scaler.fit(X_submission)

In [49]:
X_submission = scaler.transform(X_submission)

In [50]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X.shape

In [51]:
rdf = RandomForestRegressor(n_estimators=500)
rdf.fit(X,y)

In [52]:
ys_pred =rdf.predict(X_submission)
ys_pred = np.round(ys_pred)
y_pred = [0 if i < 0 else i for i in ys_pred]

In [53]:
submission = submission.drop(['player_name','player_id'],axis=1)

In [54]:
submission['Total Points'] = y_pred

In [55]:
submission.head()

In [56]:
submission.to_csv('result.csv',index=False)